In [1]:
using FITSIO
using DataFrames
using HTTP
using Interpolations
using Statistics
using Plots
using ProgressMeter
using JLD

In [2]:
function loadspec(SURVEY, RUN2D, PLATE, MJD, FIBERID)
	url = string(
		"https://dr16.sdss.org/sas/dr16/",
		SURVEY,
		"/spectro/redux/",
		RUN2D,
		"/spectra/lite/",
        string(PLATE,pad=4),
		"/spec-",
        string(PLATE,pad=4),
		"-",
		MJD,
		"-",
        string(FIBERID,pad=4),
		".fits"
	)
	return FITS(f -> DataFrame(f[2]), cacheurl(url; update_period=Inf))
end

function cacheurl(url;update_period=1)
	path = "/tmp/astro_tmp.fits"
	try
		HTTP.download(url, path; update_period)
	catch e
		ispath(path) && rm(path)
		throw(e)
	end
	return path
end

cacheurl (generic function with 1 method)

In [3]:
f_dr16 = FITS("DR16Q_v4.fits")[2]
f_plates = FITS("plates-dr16.fits")[2]
df_dr16 = DataFrame(f_dr16);
df_plates = DataFrame(f_plates);
dr16qp = innerjoin(df_dr16,df_plates; on=[:PLATE,:MJD], validate=(false,true), makeunique=true);

In [8]:
list = filter(row -> (row.SURVEY=="eboss" && row.PLATEQUALITY == "good" && (2.0 < row.Z < 2.1) && row.BAL_PROB > 0.8),dr16qp)

Row,SDSS_NAME,RA,DEC,PLATE,MJD,FIBERID,AUTOCLASS_PQN,AUTOCLASS_DR14Q,IS_QSO_QN,Z_QN,RANDOM_SELECT,Z_10K,Z_CONF_10K,PIPE_CORR_10K,IS_QSO_10K,THING_ID,Z_VI,Z_CONF,CLASS_PERSON,Z_DR12Q,IS_QSO_DR12Q,Z_DR7Q_SCH,IS_QSO_DR7Q,Z_DR6Q_HW,Z_DR7Q_HW,IS_QSO_FINAL,Z,SOURCE_Z,Z_PIPE,ZWARNING,OBJID,Z_PCA,ZWARN_PCA,DELTACHI2_PCA,Z_HALPHA,ZWARN_HALPHA,DELTACHI2_HALPHA,Z_HBETA,ZWARN_HBETA,DELTACHI2_HBETA,Z_MGII,ZWARN_MGII,DELTACHI2_MGII,Z_CIII,ZWARN_CIII,DELTACHI2_CIII,Z_CIV,ZWARN_CIV,DELTACHI2_CIV,Z_LYA,ZWARN_LYA,DELTACHI2_LYA,Z_LYAWG,Z_DLA,NHI_DLA,CONF_DLA,BAL_PROB,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,BI_SIIV,ERR_BI_SIIV,AI_SIIV,ERR_AI_SIIV,BOSS_TARGET1,EBOSS_TARGET0,EBOSS_TARGET1,EBOSS_TARGET2,ANCILLARY_TARGET1,ANCILLARY_TARGET2,NSPEC_SDSS,NSPEC_BOSS,NSPEC,PLATE_DUPLICATE,MJD_DUPLICATE,FIBERID_DUPLICATE,SPECTRO_DUPLICATE,SKYVERSION,RUN_NUMBER,RERUN_NUMBER,CAMCOL_NUMBER,FIELD_NUMBER,ID_NUMBER,LAMBDA_EFF,ZOFFSET,XFOCAL,YFOCAL,CHUNK,TILE,PLATESN2,PSFFLUX,PSFFLUX_IVAR,PSFMAG,PSFMAGERR,EXTINCTION,M_I,SN_MEDIAN_ALL,GALEX_MATCHED,FUV,⋯
,String,Float64,Float64,Int32,Int32,Int16,String,String,Int16,Float64,Int16,Float64,Int16,Int16,Int16,Int64,Float64,Int16,Int16,Float64,Int16,Float64,Int16,Float64,Float64,Int16,Float64,String,Float64,Int32,String,Float64,Int64,Float64,Float64,Int64,Float64,Float64,Int64,Float64,Float64,Int64,Float64,Float64,Int64,Float64,Float64,Int64,Float64,Float64,Int64,Float64,Float32,Array…,Array…,Array…,Float32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int32,Int32,Int32,Array…,Array…,Array…,Array…,UInt8,Int32,String,Int32,Int32,Int32,Float64,Float64,Float64,Float64,String,Int32,Float64,Array…,Array…,Array…,Array…,Array…,Float64,Float64,Int16,Float64,⋯
1,000000.33+310325.3,0.00141451,31.057,7749,58073,660,QSO,QSO,1,2.02602,0,-1.0,-1,-1,-1,376953805,-1.0,0,0,-1.0,0,-1.0,-1,-1.0,-1.0,1,2.03549,PIPE,2.03549,0,1237663234451309152,2.03444,0,522.757,-1.0,7682,0.0,-1.0,7682,0.0,2.05171,0,36.6511,2.01407,0,17.7795,2.00791,0,94.9349,2.03794,0,59.6907,2.03083,"[-1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0]",0.9,0.0,0.0,143.466,6.53822,0.0,0.0,0.0,0.0,0,0,1024,0,0,0,0,0,0,"Int32[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1 … -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","Int32[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1 … -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","Int16[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1 … -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","Int16[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1 … -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",2,4135,301,4,264,608,5400.0,0.0,-88.1344,15.4177,eboss1,16245,13.6333,"Float32[1.45554, 1.92406, 2.36621, 2.84759, 4.10104]","[28.0965, 148.0, 81.8714, 39.2631, 1.79681]","Float32[22.0825, 21.7871, 21.5621, 21.3595, 20.934]","[0.138192, 0.0461832, 0.0504525, 0.0603687, 0.185778]","Float32[0.198869, 0.154957, 0.107199, 0.0796602, 0.0592525]",-24.5386,2.28859,1,0.279323,⋯
2,000004.10+335133.0,0.0171179,33.8592,7750,58402,217,QSO,QSO,1,2.01072,0,-1.0,-1,-1,-1,395496606,-1.0,0,0,-1.0,0,-1.0,-1,-1.0,-1.0,1,2.00675,PIPE,2.00675,0,1237666183500202310,2.02371,0,13969.4,-1.0,7682,0.0,-1.0,7682,0.0,2.02215,0,337.563,2.01025,0,118.46,2.02334,0,2694.75,2.02743,0,404.451,2.0245,"[-1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0]",1.0,4139.8,9.52574,5051.21,10.3403,0.0,0.0,0.0,0.0,0,0,1024,0,0,0,0,0,0,"Int32[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1 … -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","Int32[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1 … -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","Int16[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1 … -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","Int16[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1 … -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",2,4822,301,1,523,326,5400.0,0.0,75.104,-254.787,eboss1,16248,15.4951,"Float32[10.2126, 18.4486, 30.5038, 41.7826, 53.5382]","[8.18846, 8.69243, 4.97337, 3.06016, 0.466829]","Float32[19.977, 19.3351, 18.7891, 18.4475, 18.1781]","[0.0371386, 

In [9]:
specs = []
@showprogress dt=1 desc="Gathering data..." for i = 1:nrow(list)
    pt = list[i,:]
    df = loadspec(pt.SURVEY,pt.RUN2D,pt.PLATE,pt.MJD,pt.FIBERID)
    spec = (;
			WAVE = 10.0 .^ df.LOGLAM ./ (1+pt.Z),   # Restframe wavelength
			FLUX = df.FLUX,                      # Flux
			IVAR = df.IVAR,                      # Inverse variance
		)
    push!(specs,spec)
end

Gathering data... 100%|██████████████████████████████████| Time: 0:12:39


In [11]:
WAVE = 1480:0.5:1620 #2762:0.5:2852 #1480:0.5:1620

specs2 = filter(row -> ( !isnothing(findlast(<(minimum(WAVE)),row.WAVE)) && !isnothing(findfirst(>(maximum(WAVE)),row.WAVE)) 
        && !any(iszero,row.IVAR[findlast(<(minimum(WAVE)),row.WAVE):findfirst(>(maximum(WAVE)),row.WAVE)]) ),specs);nothing

In [12]:
normspecs = []
for i = 1:length(specs2)
    spec = specs2[i]
    # Form data vectors via linear interpolation
	FLUX = LinearInterpolation(spec.WAVE, spec.FLUX).(WAVE)
	VAR  = LinearInterpolation(spec.WAVE, inv.(spec.IVAR)).(WAVE)
	# Center
	FLUX = FLUX .- mean(FLUX)
	# Normalize
	scaling = abs(mean(FLUX[1525 .< WAVE .< 1575])) # 1525 .< WAVE .< 1575 # 2780 .< WAVE .< 2832
	FLUX = FLUX / scaling
	VAR  = VAR  / scaling^2

    push!(normspecs,(;FLUX, VAR))
end
normspecs = filter(spec -> minimum(spec.VAR)/maximum(spec.VAR) > 0.4, normspecs)
normspecs = sort(normspecs; by = spec->mean(spec.VAR));

In [13]:
# Form data matrix and variance vector
Yfull = mapreduce(x -> x.FLUX, hcat, normspecs)
vfull = map(x -> mean(x.VAR), normspecs);

In [14]:
save("astroData_z2_bal.jld", "Yfull", Yfull, "vfull", vfull)